## This demo app shows:
* How to run Llama2 in the cloud hosted on Replicate.
* How to use LangChain to ask Llama general questions and follow up questions.
* How to use LangChain to load a recent PDF doc - the Llama2 paper pdf - and chat about it. This is the well known RAG (Retrieval Augmented Generation) method to let LLM such as Llama2 be able to answer questions about the data not publicly available when Llama2 was trained, or about your own data. RAG is one way to prevent LLM's hallucination. 
* You should also review the [HelloLlamaLocal](HelloLlamaLocal.ipynb) notebook for more information on RAG.

**Note** We will be using Replicate to run the examples here. You will need to first sign in with Replicate with your github account, then create a free API token [here](https://replicate.com/account/api-tokens) that you can use for a while. 
After the free trial ends, you will need to enter billing info to continue to use Llama2 hosted on Replicate.

We start by installing the necessary packages:
- sentence-transformers for text embeddings
- chromadb gives us database capabilities 
- langchain provides necessary RAG tools for this demo

And setting up the Replicate token.

In [ ]:
!pip install langchain replicate sentence-transformers chromadb

In [2]:
from getpass import getpass
import os

REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN


 ········


Next we call the Llama 2 model from replicate. In this example we will use the llama 2 13b chat model. You can find more Llama 2 models by searching for them on the [Replicate model explore page](https://replicate.com/explore?query=llama).
You can add them here in the format: model_name/version

In [ ]:
from langchain.llms import Replicate

llama2_13b = "meta/llama-2-13b-chat:f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d"
llm = Replicate(
    model=llama2_13b,
    model_kwargs={"temperature": 0.01, "top_p": 1, "max_new_tokens":500}
)

With the model set up, you are now ready to ask some questions. Here is an example of the simplest way to ask the model some general questions.

In [4]:
question = "who wrote the book Innovator's dilemma?"
answer = llm(question)
print(answer)

 Hello! I'd be happy to help you with your question. The book "The Innovator's Dilemma" was written by Clayton Christensen, an American author and professor at Harvard Business School. It was first published in 1997 and has since become a widely influential work on innovation and business strategy. Do you have any other questions or would you like more information on this topic?


We will then try to follow up the response with a question asking for more information on the book. 
Since the chat history not passed on Llama doesn't have the context and doesn't know this is more about the book thus it treats this as new query.

In [5]:
# chat history not passed so Llama doesn't have the context and doesn't know this is more about the book
followup = "tell me more"
followup_answer = llm(followup)
print(followup_answer)

 Hello! I'm here to assist you with any questions or tasks you may have. I can provide information on a wide range of topics, from science and history to entertainment and culture. I can also help with practical tasks such as converting units of measurement or calculating dates and times. Is there something specific you would like to know or discuss?


To get around this we will need to provide the model with history of the chat. 
To do this, we will use  [`ConversationBufferMemory`](https://python.langchain.com/docs/modules/memory/types/buffer) to pass the chat history to the model and give it the capability to handle follow up questions.

In [6]:
# using ConversationBufferMemory to pass memory (chat history) for follow up questions
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

Once this is set up, let us repeat the steps from before and ask the model a simple question.
Then we pass the question and answer back into the model for context along with the follow up question.

In [7]:
# restart from the original question
answer = conversation.predict(input=question)
print(answer)

 Ah, you're asking about "The Innovator's Dilemma," that classic book by Clayton Christensen! He's a renowned author and professor at Harvard Business School, known for his work on disruptive innovation and how established companies can struggle to adapt to new technologies and business models.

In fact, I have access to a wealth of information on this topic, as well as other areas of expertise. Would you like me to share some interesting facts or insights about Clayton Christensen or his book? For example, did you know that he coined the term


In [8]:
# pass context (previous question and answer) along with the follow up "tell me more" to Llama who now knows more of what
memory.save_context({"input": question},
                    {"output": answer})
followup_answer = conversation.predict(input=followup)
print(followup_answer)

 Sure thing! Here are some additional details about Clayton Christensen and his book "The Innovator's Dilemma":

1. The book was first published in 1997 and has since become a seminal work in the field of innovation and entrepreneurship.
2. Christensen's central argument is that successful companies often struggle to adopt new technologies and business models because they are too focused on sustaining their existing businesses. This can lead to a "dilemma" where these companies fail to innovate and eventually lose market share to newer, more ag


Next, let's explore using Llama 2 to answer questions using documents for context. 
This gives us the ability to update Llama 2's knowledge thus giving it better context without needing to finetune. 
For a more in-depth study of this, see the notebook on using Llama 2 locally [here](HelloLlamaLocal.ipynb)

We will use the PyPDFLoader to load in a pdf, in this case, the Llama 2 paper.

In [9]:

from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("https://arxiv.org/pdf/2307.09288.pdf")
docs = loader.load()

In [10]:
# check docs length and content
print(len(docs), docs[0].page_content[0:300])

77 Llama 2 : Open Foundation and Fine-Tuned Chat Models
Hugo Touvron∗Louis Martin†Kevin Stone†
Peter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra
Prajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen
Guillem Cucurull David Esiobu Jude Fernande


We need to store our documents. There are more than 30 vector stores (DBs) supported by LangChain. 
For this example we will use [Chroma](https://python.langchain.com/docs/integrations/vectorstores/chroma) which is light-weight and in memory so it's easy to get started with.
For other vector stores especially if you need to store a large amount of data - see https://python.langchain.com/docs/integrations/vectorstores

We will also import the HuggingFaceEmbeddings and RecursiveCharacterTextSplitter to assist in storing the documents.

In [11]:

from langchain.vectorstores import Chroma

# embeddings are numerical representations of the question and answer text
from langchain.embeddings import HuggingFaceEmbeddings

# use a common text splitter to split text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

To store the documents, we will need to split them into chunks using [`RecursiveCharacterTextSplitter`](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter) and create vector representations of these chunks using [`HuggingFaceEmbeddings`](https://www.google.com/search?q=langchain+hugging+face+embeddings&sca_esv=572890011&ei=ARUoZaH4LuumptQP48ah2Ac&oq=langchian+hugg&gs_lp=Egxnd3Mtd2l6LXNlcnAiDmxhbmdjaGlhbiBodWdnKgIIADIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCkjeHlC5Cli5D3ABeAGQAQCYAV6gAb4CqgEBNLgBAcgBAPgBAcICChAAGEcY1gQYsAPiAwQYACBBiAYBkAYI&sclient=gws-wiz-serp) to them before storing them into our vector database. 

In general, you should use larger chuck sizes for highly structured text such as code and smaller size for less structured text. You may need to experiment with different chunk sizes and overlap values to find out the best numbers.

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(docs)

# create the vector db to store all the split chunks as embeddings
embeddings = HuggingFaceEmbeddings()
vectordb = Chroma.from_documents(
    documents=all_splits,
    embedding=embeddings,
)

We then use ` RetrievalQA` to retrieve the documents from the vector database and give the model more context on Llama 2, thereby increasing its knowledge.
For each question, LangChain performs a semantic similarity search of it in the vector db, then passes the search results as the context to Llama to answer the question.

In [13]:
# use LangChain's RetrievalQA, to associate Llama with the loaded documents stored in the vector db
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)


question = "What is llama2?"
result = qa_chain({"query": question})
print(result['result'])

 Based on the provided text, Llama2 appears to be a language model developed by Meta AI that is designed for dialogue use cases. It is a fine-tuned version of the original Llama model, with improved performance and safety features. The model has been trained on a large dataset of text and has undergone testing in English, but it may not cover all scenarios or produce accurate responses in certain instances. As such, developers are advised to perform safety testing and tuning before deploying any applications of Llama2. Additionally, the model is released under a responsible use guide and code


Now, lets bring it all together by incorporating follow up questions.
First we ask a follow up questions without giving the model context of the previous conversation. 
Without this context, the answer we get does not relate to our original question.

In [14]:
# no context passed so Llama2 doesn't have enough context to answer so it lets its imagination go wild
result = qa_chain({"query": "what are its use cases?"})
print(result['result'])

 Based on the context provided, I don't see any explicit mention of "its" use cases. However, I can provide some possible inferences based on the information given:

The text mentions a partnerships team and product and technical organization support, which suggests that the tool or approach being referred to is likely related to product development or customer support.

The emphasis on prioritizing harmlessness over informativeness and helpfulness suggests that the tool may be used for moderation or content review purposes, where the goal is to avoid causing harm or offense while still providing useful information.

The


As we did before, let us use the ConversationalRetrievalChain package to give the model context of our previous question so we can add follow up questions.

In [15]:
# use ConversationalRetrievalChain to pass chat history for follow up questions
from langchain.chains import ConversationalRetrievalChain
chat_chain = ConversationalRetrievalChain.from_llm(llm, vectordb.as_retriever(), return_source_documents=True)

In [16]:
# let's ask the original question "What is llama2?" again
result = chat_chain({"question": question, "chat_history": []})
print(result['answer'])

 Based on the provided text, Llama2 appears to be a language model developed by Meta AI that is designed for dialogue use cases. It is a fine-tuned version of the original Llama model, with improved performance and safety features. The model has been trained on a large dataset of text and has undergone testing in English, but it may not cover all scenarios or produce accurate responses in certain instances. As such, developers are advised to perform safety testing and tuning before deploying any applications of Llama2. Additionally, the model is released under a responsible use guide and code


In [17]:
# this time we pass chat history along with the follow up so good things should happen
chat_history = [(question, result["answer"])]
followup = "what are its use cases?"
followup_answer = chat_chain({"question": followup, "chat_history": chat_history})
print(followup_answer['answer'])

 Based on the provided context, here are some potential use cases for Llama2, a language model developed by Meta AI for dialogue use cases:

1. Assistant-like chat: Tuned models of Llama2 can be used for assistant-like chat applications, such as customer service or personal assistants.
2. Natural language generation tasks: Pretrained models of Llama2 can be adapted for various natural language generation tasks, such as text summarization, machine translation, and content creation.
3. Research use cases: Llama2 can be used in research studies to


Further follow ups can be made possible by updating chat_history.
Note that results can get cut off. You may set "max_new_tokens" in the Replicate call above to a larger number (like shown below) to avoid the cut off.

```python
model_kwargs={"temperature": 0.01, "top_p": 1, "max_new_tokens": 1000}
```

In [18]:
# further follow ups can be made possible by updating chat_history like this:
chat_history.append((followup, followup_answer["answer"]))
more_followup = "what tasks can it assist with?"
more_followup_answer = chat_chain({"question": more_followup, "chat_history": chat_history})
print(more_followup_answer['answer'])



 Based on the information provided, Llama2 can assist with various natural language generation tasks, particularly in English. The model has been fine-tuned for assistant-like chat and has shown proficiency in other languages as well, although its proficiency is limited due to the limited amount of pretraining data available in non-English languages.

Specifically, Llama2 can be used for tasks such as:

1. Dialogue systems: Llama2 can be fine-tuned for different dialogue systems, such as customer service chatbots, virtual assistants,
